**Step 1: Install Dependencies**

First, we need to install the latest computer vision libraries. YOLO11 provides the detection engine, while Supervision handles the "aesthetic" annotations.

In [ ]:
!pip install -U ultralytics supervision lapx --quiet
import ultralytics
import supervision as sv
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab import drive, files
from tqdm.notebook import tqdm

print("✅ Setup Complete")

✅ Setup Complete


**Step 2: Mount Google Drive**

Since your video is in Drive, we need to grant the notebook access to your files.

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# UPDATE THESE PATHS
INPUT_VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/gameinputs/game02.mp4"
# OUTPUT_VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/gameoutputs/game02_ai.mp4"
# OUTPUT_VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/gameoutputs/game02_ai_heatvision.mp4"
OUTPUT_VIDEO_PATH = "/content/drive/MyDrive/Colab Notebooks/gameoutputs/game02_ai_line.mp4"

**Step 3: The "Hidden Segmentation"**

This script uses yolo11x-seg.pt for the math but only draws the boxes and trails for the video. I have also added a Directional Check to ensure the ball is moving downward to count as a score.

In [ ]:
# # ============================================================
# # AUTO HOOP GATES + ROCK-SOLID BALL (KALMAN + REACQUIRE) + ROBUST SCORE
# # (NO tracker_id REQUIRED) + NumPy 1.25 FIX + HOOP ROI BOOST
# # FULL SELF-CONTAINED BLOCK (includes annotators + lighting fix)
# # ============================================================

# # 0. Setup annotators (NO LABELS)
# person_ann = sv.RoundBoxAnnotator(thickness=2, color=sv.Color.RED)
# ball_ann   = sv.RoundBoxAnnotator(thickness=2, color=sv.Color.WHITE)
# trace_ann  = sv.TraceAnnotator(thickness=3, trace_length=50)

# # 1. Load Model
# model = YOLO("yolo11x-seg.pt")

# # ----------------------------
# # Lighting fix
# # ----------------------------
# def apply_lighting_fix(frame):
#     lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
#     l, a, b = cv2.split(lab)
#     clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
#     l2 = clahe.apply(l)
#     enhanced = cv2.merge((l2, a, b))
#     return cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

# # ----------------------------
# # Auto hoop gates from rim color
# # ----------------------------
# def auto_hoop_gates_from_video(video_path: str):
#     cap = cv2.VideoCapture(video_path)
#     ok, frame = cap.read()
#     cap.release()
#     if not ok or frame is None:
#         raise RuntimeError("Could not read first frame.")

#     H, W = frame.shape[:2]
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

#     lower_orange = np.array([5, 60, 60], dtype=np.uint8)
#     upper_orange = np.array([30, 255, 255], dtype=np.uint8)
#     mask = cv2.inRange(hsv, lower_orange, upper_orange)

#     top_h = int(H * 0.30)
#     mask[top_h:, :] = 0
#     mask[:top_h, :int(W * 0.35)] = 0

#     kernel = np.ones((3, 3), np.uint8)
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
#     mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel, iterations=2)

#     ys, xs = np.where(mask > 0)
#     if len(xs) < 80:
#         raise RuntimeError("Rim not detected. Widen orange HSV thresholds.")

#     x1, x2 = int(xs.min()), int(xs.max())
#     y1, y2 = int(ys.min()), int(ys.max())

#     rim_cx = (x1 + x2) / 2.0
#     if abs(rim_cx - (W / 2.0)) > (W * 0.22):
#         raise RuntimeError("Detected orange region is not near hoop center. Adjust mask exclusions.")

#     rim_w = max(1, x2 - x1)
#     pad_x = int(max(10, rim_w * 0.20))

#     ux1 = max(0, x1 - pad_x)
#     ux2 = min(W - 1, x2 + pad_x)

#     uy1 = max(0, y1 - 8)
#     uy2 = min(H - 1, y2 + 18)

#     ly1 = min(H - 1, y2 + 5)
#     ly2 = min(H - 1, y2 + 125)

#     upper_poly = np.array([[ux1, uy1], [ux2, uy1], [ux2, uy2], [ux1, uy2]], dtype=int)
#     lower_poly = np.array([[ux1, ly1], [ux2, ly1], [ux2, ly2], [ux1, ly2]], dtype=int)
#     return upper_poly, lower_poly

# video_info = sv.VideoInfo.from_video_path(INPUT_VIDEO_PATH)

# try:
#     HOOP_UPPER_POLYGON, HOOP_LOWER_POLYGON = auto_hoop_gates_from_video(INPUT_VIDEO_PATH)
#     print("✅ Auto hoop gates detected:")
#     print("UPPER:", HOOP_UPPER_POLYGON.tolist())
#     print("LOWER:", HOOP_LOWER_POLYGON.tolist())
# except Exception as e:
#     print("⚠️ Auto hoop gates failed, using fallback. Error:", str(e))
#     HOOP_UPPER_POLYGON = np.array([[140, 0], [330, 0], [330, 95], [140, 95]], dtype=int)
#     HOOP_LOWER_POLYGON = np.array([[150, 85], [320, 85], [320, 215], [150, 215]], dtype=int)

# upper_zone = sv.PolygonZone(polygon=HOOP_UPPER_POLYGON)
# lower_zone = sv.PolygonZone(polygon=HOOP_LOWER_POLYGON)

# # ----------------------------
# # Ball stabilizer (Kalman filter)
# # ----------------------------
# class BallTrack:
#     def __init__(self):
#         self.kf = cv2.KalmanFilter(4, 2)
#         self.kf.transitionMatrix = np.array(
#             [[1, 0, 1, 0],
#              [0, 1, 0, 1],
#              [0, 0, 1, 0],
#              [0, 0, 0, 1]], dtype=np.float32
#         )
#         self.kf.measurementMatrix = np.array(
#             [[1, 0, 0, 0],
#              [0, 1, 0, 0]], dtype=np.float32
#         )
#         self.kf.processNoiseCov = np.eye(4, dtype=np.float32) * 1e-2
#         self.kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 2.5e-1
#         self.kf.errorCovPost = np.eye(4, dtype=np.float32)

#         self.initialized = False
#         self.missed = 0
#         self.w = 24.0
#         self.h = 24.0

#     def init(self, cx, cy, w, h):
#         self.kf.statePost = np.array([[cx], [cy], [0.0], [0.0]], dtype=np.float32)
#         self.kf.statePre = self.kf.statePost.copy()
#         self.w = float(w)
#         self.h = float(h)
#         self.initialized = True
#         self.missed = 0

#     def predict(self):
#         if not self.initialized:
#             return None
#         pred = self.kf.predict()
#         return float(pred[0, 0]), float(pred[1, 0])

#     def update(self, cx, cy, w=None, h=None):
#         m = np.array([[cx], [cy]], dtype=np.float32)
#         self.kf.correct(m)
#         if w is not None and h is not None:
#             self.w = 0.85 * self.w + 0.15 * float(w)
#             self.h = 0.85 * self.h + 0.15 * float(h)
#         self.missed = 0

#     def mark_missed(self):
#         if self.initialized:
#             self.missed += 1

#     def bbox(self):
#         if not self.initialized:
#             return None
#         cx = float(self.kf.statePost[0, 0])
#         cy = float(self.kf.statePost[1, 0])
#         x1 = cx - self.w / 2.0
#         y1 = cy - self.h / 2.0
#         x2 = cx + self.w / 2.0
#         y2 = cy + self.h / 2.0
#         return np.array([x1, y1, x2, y2], dtype=np.float32)

# def det_centroid_wh(dets: sv.Detections):
#     xyxy = dets.xyxy
#     cx = (xyxy[:, 0] + xyxy[:, 2]) / 2.0
#     cy = (xyxy[:, 1] + xyxy[:, 3]) / 2.0
#     w = (xyxy[:, 2] - xyxy[:, 0])
#     h = (xyxy[:, 3] - xyxy[:, 1])
#     return cx, cy, w, h

# def pick_best_ball_near_prediction(dets_ball: sv.Detections, pred_xy, max_dist=140):
#     if dets_ball is None or len(dets_ball) == 0:
#         return None

#     if pred_xy is None:
#         idx = int(np.argmax(dets_ball.confidence)) if dets_ball.confidence is not None else 0
#         return dets_ball[[idx]]

#     px, py = pred_xy
#     cx, cy, _, _ = det_centroid_wh(dets_ball)
#     d = np.sqrt((cx - px) ** 2 + (cy - py) ** 2)
#     idx = int(np.argmin(d))

#     if float(d[idx]) <= max_dist:
#         return dets_ball[[idx]]

#     idx2 = int(np.argmax(dets_ball.confidence)) if dets_ball.confidence is not None else 0
#     return dets_ball[[idx2]]

# def clamp_xyxy(xyxy, W, H):
#     x1, y1, x2, y2 = map(float, xyxy.tolist())
#     x1 = max(0, min(W - 1, x1))
#     y1 = max(0, min(H - 1, y1))
#     x2 = max(0, min(W - 1, x2))
#     y2 = max(0, min(H - 1, y2))
#     return np.array([x1, y1, x2, y2], dtype=np.float32)

# def union_gate_roi(upper_poly, lower_poly, W, H, pad=20):
#     xs = np.concatenate([upper_poly[:, 0], lower_poly[:, 0]])
#     ys = np.concatenate([upper_poly[:, 1], lower_poly[:, 1]])
#     x1 = max(0, int(xs.min()) - pad)
#     y1 = max(0, int(ys.min()) - pad)
#     x2 = min(W - 1, int(xs.max()) + pad)
#     y2 = min(H - 1, int(ys.max()) + pad)
#     return x1, y1, x2, y2

# def offset_detections(dets: sv.Detections, dx: int, dy: int):
#     if dets is None or len(dets) == 0:
#         return dets
#     xyxy = dets.xyxy.copy()
#     xyxy[:, [0, 2]] += dx
#     xyxy[:, [1, 3]] += dy
#     return sv.Detections(
#         xyxy=xyxy,
#         confidence=dets.confidence.copy() if dets.confidence is not None else None,
#         class_id=dets.class_id.copy() if dets.class_id is not None else None,
#         tracker_id=dets.tracker_id.copy() if dets.tracker_id is not None else None,
#     )

# def concat_dets(a: sv.Detections, b: sv.Detections):
#     if a is None or len(a) == 0:
#         return b
#     if b is None or len(b) == 0:
#         return a
#     return sv.Detections(
#         xyxy=np.concatenate([a.xyxy, b.xyxy], axis=0),
#         confidence=np.concatenate([a.confidence, b.confidence], axis=0)
#         if a.confidence is not None and b.confidence is not None else None,
#         class_id=np.concatenate([a.class_id, b.class_id], axis=0)
#         if a.class_id is not None and b.class_id is not None else None,
#         tracker_id=None,
#     )

# # -----------------------
# # Robust scoring state
# # -----------------------
# score = 0
# COOLDOWN_FRAMES = 45
# MAX_TRANSIT_FRAMES = 26
# MIN_DOWN_PIXELS = 24
# MISS_RESET_FRAMES = 20

# cooldown = 0
# attempt = {"phase": "idle", "start_frame": None, "start_cy": None}

# ball_track = BallTrack()
# MAX_MISSED_FRAMES = 30

# dbg_upper_hits = 0
# dbg_lower_hits = 0
# dbg_scores = 0

# # -----------------------
# # Continuous person boxes + trail even when Ultralytics tracker ids disappear
# # (annotation-only lightweight tracker)
# # -----------------------
# PERSON_MAX_MISSED = 18
# PERSON_MATCH_IOU = 0.18

# def _iou_xyxy(a, b):
#     ax1, ay1, ax2, ay2 = map(float, a)
#     bx1, by1, bx2, by2 = map(float, b)
#     ix1 = max(ax1, bx1)
#     iy1 = max(ay1, by1)
#     ix2 = min(ax2, bx2)
#     iy2 = min(ay2, by2)
#     iw = max(0.0, ix2 - ix1)
#     ih = max(0.0, iy2 - iy1)
#     inter = iw * ih
#     area_a = max(0.0, (ax2 - ax1)) * max(0.0, (ay2 - ay1))
#     area_b = max(0.0, (bx2 - bx1)) * max(0.0, (by2 - by1))
#     denom = area_a + area_b - inter
#     return (inter / denom) if denom > 1e-6 else 0.0

# class _PersonAnnotTracker:
#     def __init__(self):
#         self.next_id = 1
#         self.tracks = {}  # id -> {"xyxy": np.array(4), "missed": int}

#     def update(self, person_xyxy: np.ndarray):
#         for tid in list(self.tracks.keys()):
#             self.tracks[tid]["missed"] += 1

#         if person_xyxy is None or len(person_xyxy) == 0:
#             for tid in list(self.tracks.keys()):
#                 if self.tracks[tid]["missed"] > PERSON_MAX_MISSED:
#                     self.tracks.pop(tid, None)
#             return

#         used_det = set()
#         track_ids = list(self.tracks.keys())

#         for tid in track_ids:
#             best_j = -1
#             best_iou = 0.0
#             tbox = self.tracks[tid]["xyxy"]
#             for j in range(len(person_xyxy)):
#                 if j in used_det:
#                     continue
#                 iou = _iou_xyxy(tbox, person_xyxy[j])
#                 if iou > best_iou:
#                     best_iou = iou
#                     best_j = j
#             if best_j >= 0 and best_iou >= PERSON_MATCH_IOU:
#                 self.tracks[tid]["xyxy"] = person_xyxy[best_j].astype(np.float32)
#                 self.tracks[tid]["missed"] = 0
#                 used_det.add(best_j)

#         for j in range(len(person_xyxy)):
#             if j in used_det:
#                 continue
#             tid = self.next_id
#             self.next_id += 1
#             self.tracks[tid] = {"xyxy": person_xyxy[j].astype(np.float32), "missed": 0}

#         for tid in list(self.tracks.keys()):
#             if self.tracks[tid]["missed"] > PERSON_MAX_MISSED:
#                 self.tracks.pop(tid, None)

#     def as_detections(self):
#         if len(self.tracks) == 0:
#             return None
#         tids = sorted(self.tracks.keys())
#         xyxy = np.stack([self.tracks[tid]["xyxy"] for tid in tids], axis=0).astype(np.float32)
#         return sv.Detections(
#             xyxy=xyxy,
#             confidence=np.ones((len(tids),), dtype=np.float32),
#             class_id=np.zeros((len(tids),), dtype=np.int32),
#             tracker_id=np.array(tids, dtype=np.int32),
#         )

# person_annot_tracker = _PersonAnnotTracker()

# # 5. Processing Loop
# frames_generator = sv.get_video_frames_generator(source_path=INPUT_VIDEO_PATH)

# with sv.VideoSink(target_path=OUTPUT_VIDEO_PATH, video_info=video_info) as sink:
#     frame_idx = -1

#     for frame in tqdm(frames_generator, total=video_info.total_frames, desc="Master Logic Render"):
#         frame_idx += 1

#         enhanced_frame = apply_lighting_fix(frame)
#         H, W = enhanced_frame.shape[:2]

#         results = model.track(
#             enhanced_frame,
#             persist=True,
#             verbose=False,
#             conf=0.03,
#             iou=0.4,
#             tracker="botsort.yaml"
#         )[0]

#         detections = sv.Detections.from_ultralytics(results)
#         detections = detections[(detections.class_id == 0) | (detections.class_id == 32)]

#         # HOOP ROI BOOST
#         rx1, ry1, rx2, ry2 = union_gate_roi(HOOP_UPPER_POLYGON, HOOP_LOWER_POLYGON, W, H, pad=30)
#         roi = enhanced_frame[ry1:ry2, rx1:rx2]

#         roi_ball = None
#         if roi.size > 0:
#             roi_res = model.predict(roi, verbose=False, conf=0.01, iou=0.35)[0]
#             roi_det = sv.Detections.from_ultralytics(roi_res)
#             roi_det = roi_det[roi_det.class_id == 32]
#             roi_ball = offset_detections(roi_det, dx=rx1, dy=ry1)

#         full_ball = detections[detections.class_id == 32]
#         merged_ball = concat_dets(full_ball, roi_ball)

#         # Ball: predict -> match -> update
#         pred_xy = ball_track.predict()
#         chosen = pick_best_ball_near_prediction(merged_ball, pred_xy, max_dist=160)

#         if chosen is not None and len(chosen) > 0:
#             cx, cy, bw, bh = det_centroid_wh(chosen)
#             cx = float(cx[0]); cy = float(cy[0])
#             bw = float(bw[0]); bh = float(bh[0])

#             if not ball_track.initialized:
#                 ball_track.init(cx, cy, bw, bh)
#             else:
#                 ball_track.update(cx, cy, bw, bh)
#         else:
#             ball_track.mark_missed()

#         # Virtual ball from Kalman
#         ball_for_zone = None
#         ball_cy = None

#         if ball_track.initialized and ball_track.missed <= MAX_MISSED_FRAMES:
#             bb = ball_track.bbox()
#             if bb is not None:
#                 ball_xyxy = clamp_xyxy(bb, W, H)
#                 ball_cy = float((ball_xyxy[1] + ball_xyxy[3]) / 2.0)
#                 ball_for_zone = sv.Detections(
#                     xyxy=np.array([ball_xyxy], dtype=np.float32),
#                     confidence=np.array([1.0], dtype=np.float32),
#                     class_id=np.array([32], dtype=np.int32),
#                 )
#         else:
#             ball_track.initialized = False

#         ball_seen = ball_for_zone is not None
#         in_upper = upper_zone.trigger(detections=ball_for_zone).any() if ball_seen else False
#         in_lower = lower_zone.trigger(detections=ball_for_zone).any() if ball_seen else False

#         if in_upper:
#             dbg_upper_hits += 1
#         if in_lower:
#             dbg_lower_hits += 1

#         if cooldown > 0:
#             cooldown -= 1

#         # Attempt state machine
#         if attempt["phase"] == "idle":
#             if ball_seen and in_upper and cooldown == 0:
#                 attempt["phase"] = "saw_upper"
#                 attempt["start_frame"] = frame_idx
#                 attempt["start_cy"] = ball_cy

#         elif attempt["phase"] == "saw_upper":
#             if (frame_idx - attempt["start_frame"]) > MAX_TRANSIT_FRAMES:
#                 attempt["phase"] = "idle"
#                 attempt["start_frame"] = None
#                 attempt["start_cy"] = None

#             elif ball_seen and in_lower and cooldown == 0:
#                 down_pixels = ball_cy - float(attempt["start_cy"])
#                 if down_pixels >= MIN_DOWN_PIXELS:
#                     score += 1
#                     cooldown = COOLDOWN_FRAMES
#                     dbg_scores += 1

#                 attempt["phase"] = "idle"
#                 attempt["start_frame"] = None
#                 attempt["start_cy"] = None

#             elif (not ball_seen) and (frame_idx - attempt["start_frame"]) > MISS_RESET_FRAMES:
#                 attempt["phase"] = "idle"
#                 attempt["start_frame"] = None
#                 attempt["start_cy"] = None

#         # Annotation
#         annotated_frame = frame.copy()

#         # Always draw persons, even if Ultralytics tracker ids disappear after scene shifts
#         p_xyxy = detections[detections.class_id == 0].xyxy if len(detections) > 0 else np.zeros((0, 4), dtype=np.float32)
#         if p_xyxy is None:
#             p_xyxy = np.zeros((0, 4), dtype=np.float32)

#         person_annot_tracker.update(p_xyxy)
#         p_tracked = person_annot_tracker.as_detections()

#         if p_tracked is not None and len(p_tracked) > 0:
#             annotated_frame = trace_ann.annotate(scene=annotated_frame, detections=p_tracked)
#             annotated_frame = person_ann.annotate(scene=annotated_frame, detections=p_tracked)

#         if ball_for_zone is not None:
#             annotated_frame = ball_ann.annotate(scene=annotated_frame, detections=ball_for_zone)

#         cv2.polylines(annotated_frame, [HOOP_UPPER_POLYGON], True, (0, 255, 255), 2)
#         cv2.polylines(annotated_frame, [HOOP_LOWER_POLYGON], True, (255, 255, 0), 2)

#         # Scoreboard
#         h, w, _ = annotated_frame.shape
#         hud_w, margin = 320, 40
#         overlay = annotated_frame.copy()
#         cv2.rectangle(overlay, (w - hud_w - margin, h - 120), (w - margin, h - 40), (15, 15, 15), -1)
#         cv2.rectangle(overlay, (w - hud_w - margin, h - 120), (w - hud_w - margin + 10, h - 40), (0, 255, 0), -1)
#         annotated_frame = cv2.addWeighted(overlay, 0.7, annotated_frame, 0.3, 0)

#         cv2.putText(annotated_frame, "BASKETBALL PERFORMANCE", (w - hud_w - margin + 25, h - 95),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.4, (160, 160, 160), 1, cv2.LINE_AA)
#         cv2.putText(annotated_frame, f"PTS: {score}", (w - hud_w - margin + 25, h - 55),
#                     cv2.FONT_HERSHEY_DUPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

#         sink.write_frame(frame=annotated_frame)

# print(f"\n✅ Render Complete! AI Counted: {score}.")
# print("Debug upper hits:", dbg_upper_hits, "lower hits:", dbg_lower_hits, "scores:", dbg_scores)

In [ ]:
# # ============================================================
# # AUTO HOOP GATES + ROCK-SOLID BALL (KALMAN + REACQUIRE) + ROBUST SCORE
# # (NO tracker_id REQUIRED) + NumPy 1.25 FIX + HOOP ROI BOOST
# # FULL SELF-CONTAINED BLOCK (includes annotators + lighting fix)
# # ============================================================

# # 0. Setup annotators (NO LABELS)
# person_ann = sv.RoundBoxAnnotator(thickness=2, color=sv.Color.RED)
# ball_ann   = sv.RoundBoxAnnotator(thickness=2, color=sv.Color.WHITE)
# trace_ann  = sv.TraceAnnotator(thickness=3, trace_length=50)

# # 1. Load Model
# model = YOLO("yolo11x-seg.pt")

# # ----------------------------
# # Lighting fix
# # ----------------------------
# def apply_lighting_fix(frame):
#     lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
#     l, a, b = cv2.split(lab)
#     clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
#     l2 = clahe.apply(l)
#     enhanced = cv2.merge((l2, a, b))
#     return cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

# # ----------------------------
# # Auto hoop gates from rim color
# # ----------------------------
# def auto_hoop_gates_from_video(video_path: str):
#     cap = cv2.VideoCapture(video_path)
#     ok, frame = cap.read()
#     cap.release()
#     if not ok or frame is None:
#         raise RuntimeError("Could not read first frame.")

#     H, W = frame.shape[:2]
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

#     lower_orange = np.array([5, 60, 60], dtype=np.uint8)
#     upper_orange = np.array([30, 255, 255], dtype=np.uint8)
#     mask = cv2.inRange(hsv, lower_orange, upper_orange)

#     top_h = int(H * 0.30)
#     mask[top_h:, :] = 0
#     mask[:top_h, :int(W * 0.35)] = 0

#     kernel = np.ones((3, 3), np.uint8)
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
#     mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel, iterations=2)

#     ys, xs = np.where(mask > 0)
#     if len(xs) < 80:
#         raise RuntimeError("Rim not detected. Widen orange HSV thresholds.")

#     x1, x2 = int(xs.min()), int(xs.max())
#     y1, y2 = int(ys.min()), int(ys.max())

#     rim_cx = (x1 + x2) / 2.0
#     if abs(rim_cx - (W / 2.0)) > (W * 0.22):
#         raise RuntimeError("Detected orange region is not near hoop center. Adjust mask exclusions.")

#     rim_w = max(1, x2 - x1)
#     pad_x = int(max(10, rim_w * 0.20))

#     ux1 = max(0, x1 - pad_x)
#     ux2 = min(W - 1, x2 + pad_x)

#     uy1 = max(0, y1 - 8)
#     uy2 = min(H - 1, y2 + 18)

#     ly1 = min(H - 1, y2 + 5)
#     ly2 = min(H - 1, y2 + 125)

#     upper_poly = np.array([[ux1, uy1], [ux2, uy1], [ux2, uy2], [ux1, uy2]], dtype=int)
#     lower_poly = np.array([[ux1, ly1], [ux2, ly1], [ux2, ly2], [ux1, ly2]], dtype=int)
#     return upper_poly, lower_poly

# video_info = sv.VideoInfo.from_video_path(INPUT_VIDEO_PATH)

# try:
#     HOOP_UPPER_POLYGON, HOOP_LOWER_POLYGON = auto_hoop_gates_from_video(INPUT_VIDEO_PATH)
#     print("✅ Auto hoop gates detected:")
#     print("UPPER:", HOOP_UPPER_POLYGON.tolist())
#     print("LOWER:", HOOP_LOWER_POLYGON.tolist())
# except Exception as e:
#     print("⚠️ Auto hoop gates failed, using fallback. Error:", str(e))
#     HOOP_UPPER_POLYGON = np.array([[140, 0], [330, 0], [330, 95], [140, 95]], dtype=int)
#     HOOP_LOWER_POLYGON = np.array([[150, 85], [320, 85], [320, 215], [150, 215]], dtype=int)

# upper_zone = sv.PolygonZone(polygon=HOOP_UPPER_POLYGON)
# lower_zone = sv.PolygonZone(polygon=HOOP_LOWER_POLYGON)

# # ----------------------------
# # Ball stabilizer (Kalman filter)
# # ----------------------------
# class BallTrack:
#     def __init__(self):
#         self.kf = cv2.KalmanFilter(4, 2)
#         self.kf.transitionMatrix = np.array(
#             [[1, 0, 1, 0],
#              [0, 1, 0, 1],
#              [0, 0, 1, 0],
#              [0, 0, 0, 1]], dtype=np.float32
#         )
#         self.kf.measurementMatrix = np.array(
#             [[1, 0, 0, 0],
#              [0, 1, 0, 0]], dtype=np.float32
#         )
#         self.kf.processNoiseCov = np.eye(4, dtype=np.float32) * 1e-2
#         self.kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 2.5e-1
#         self.kf.errorCovPost = np.eye(4, dtype=np.float32)

#         self.initialized = False
#         self.missed = 0
#         self.w = 24.0
#         self.h = 24.0

#     def init(self, cx, cy, w, h):
#         self.kf.statePost = np.array([[cx], [cy], [0.0], [0.0]], dtype=np.float32)
#         self.kf.statePre = self.kf.statePost.copy()
#         self.w = float(w)
#         self.h = float(h)
#         self.initialized = True
#         self.missed = 0

#     def predict(self):
#         if not self.initialized:
#             return None
#         pred = self.kf.predict()
#         return float(pred[0, 0]), float(pred[1, 0])

#     def update(self, cx, cy, w=None, h=None):
#         m = np.array([[cx], [cy]], dtype=np.float32)
#         self.kf.correct(m)
#         if w is not None and h is not None:
#             self.w = 0.85 * self.w + 0.15 * float(w)
#             self.h = 0.85 * self.h + 0.15 * float(h)
#         self.missed = 0

#     def mark_missed(self):
#         if self.initialized:
#             self.missed += 1

#     def bbox(self):
#         if not self.initialized:
#             return None
#         cx = float(self.kf.statePost[0, 0])
#         cy = float(self.kf.statePost[1, 0])
#         x1 = cx - self.w / 2.0
#         y1 = cy - self.h / 2.0
#         x2 = cx + self.w / 2.0
#         y2 = cy + self.h / 2.0
#         return np.array([x1, y1, x2, y2], dtype=np.float32)

# def det_centroid_wh(dets: sv.Detections):
#     xyxy = dets.xyxy
#     cx = (xyxy[:, 0] + xyxy[:, 2]) / 2.0
#     cy = (xyxy[:, 1] + xyxy[:, 3]) / 2.0
#     w = (xyxy[:, 2] - xyxy[:, 0])
#     h = (xyxy[:, 3] - xyxy[:, 1])
#     return cx, cy, w, h

# def pick_best_ball_near_prediction(dets_ball: sv.Detections, pred_xy, max_dist=140):
#     if dets_ball is None or len(dets_ball) == 0:
#         return None

#     if pred_xy is None:
#         idx = int(np.argmax(dets_ball.confidence)) if dets_ball.confidence is not None else 0
#         return dets_ball[[idx]]

#     px, py = pred_xy
#     cx, cy, _, _ = det_centroid_wh(dets_ball)
#     d = np.sqrt((cx - px) ** 2 + (cy - py) ** 2)
#     idx = int(np.argmin(d))

#     if float(d[idx]) <= max_dist:
#         return dets_ball[[idx]]

#     idx2 = int(np.argmax(dets_ball.confidence)) if dets_ball.confidence is not None else 0
#     return dets_ball[[idx2]]

# def clamp_xyxy(xyxy, W, H):
#     x1, y1, x2, y2 = map(float, xyxy.tolist())
#     x1 = max(0, min(W - 1, x1))
#     y1 = max(0, min(H - 1, y1))
#     x2 = max(0, min(W - 1, x2))
#     y2 = max(0, min(H - 1, y2))
#     return np.array([x1, y1, x2, y2], dtype=np.float32)

# def union_gate_roi(upper_poly, lower_poly, W, H, pad=20):
#     xs = np.concatenate([upper_poly[:, 0], lower_poly[:, 0]])
#     ys = np.concatenate([upper_poly[:, 1], lower_poly[:, 1]])
#     x1 = max(0, int(xs.min()) - pad)
#     y1 = max(0, int(ys.min()) - pad)
#     x2 = min(W - 1, int(xs.max()) + pad)
#     y2 = min(H - 1, int(ys.max()) + pad)
#     return x1, y1, x2, y2

# def offset_detections(dets: sv.Detections, dx: int, dy: int):
#     if dets is None or len(dets) == 0:
#         return dets
#     xyxy = dets.xyxy.copy()
#     xyxy[:, [0, 2]] += dx
#     xyxy[:, [1, 3]] += dy
#     return sv.Detections(
#         xyxy=xyxy,
#         confidence=dets.confidence.copy() if dets.confidence is not None else None,
#         class_id=dets.class_id.copy() if dets.class_id is not None else None,
#         tracker_id=dets.tracker_id.copy() if dets.tracker_id is not None else None,
#     )

# def concat_dets(a: sv.Detections, b: sv.Detections):
#     if a is None or len(a) == 0:
#         return b
#     if b is None or len(b) == 0:
#         return a
#     return sv.Detections(
#         xyxy=np.concatenate([a.xyxy, b.xyxy], axis=0),
#         confidence=np.concatenate([a.confidence, b.confidence], axis=0)
#         if a.confidence is not None and b.confidence is not None else None,
#         class_id=np.concatenate([a.class_id, b.class_id], axis=0)
#         if a.class_id is not None and b.class_id is not None else None,
#         tracker_id=None,
#     )

# # -----------------------
# # Robust scoring state
# # -----------------------
# score = 0
# COOLDOWN_FRAMES = 45
# MAX_TRANSIT_FRAMES = 26
# MIN_DOWN_PIXELS = 24
# MISS_RESET_FRAMES = 20

# cooldown = 0
# attempt = {"phase": "idle", "start_frame": None, "start_cy": None}

# ball_track = BallTrack()
# MAX_MISSED_FRAMES = 30

# dbg_upper_hits = 0
# dbg_lower_hits = 0
# dbg_scores = 0

# # -----------------------
# # Continuous person boxes + trail even when Ultralytics tracker ids disappear
# # (annotation-only lightweight tracker)
# # -----------------------
# PERSON_MAX_MISSED = 18
# PERSON_MATCH_IOU = 0.18

# def _iou_xyxy(a, b):
#     ax1, ay1, ax2, ay2 = map(float, a)
#     bx1, by1, bx2, by2 = map(float, b)
#     ix1 = max(ax1, bx1)
#     iy1 = max(ay1, by1)
#     ix2 = min(ax2, bx2)
#     iy2 = min(ay2, by2)
#     iw = max(0.0, ix2 - ix1)
#     ih = max(0.0, iy2 - iy1)
#     inter = iw * ih
#     area_a = max(0.0, (ax2 - ax1)) * max(0.0, (ay2 - ay1))
#     area_b = max(0.0, (bx2 - bx1)) * max(0.0, (by2 - by1))
#     denom = area_a + area_b - inter
#     return (inter / denom) if denom > 1e-6 else 0.0

# class _PersonAnnotTracker:
#     def __init__(self):
#         self.next_id = 1
#         self.tracks = {}  # id -> {"xyxy": np.array(4), "missed": int}

#     def update(self, person_xyxy: np.ndarray):
#         for tid in list(self.tracks.keys()):
#             self.tracks[tid]["missed"] += 1

#         if person_xyxy is None or len(person_xyxy) == 0:
#             for tid in list(self.tracks.keys()):
#                 if self.tracks[tid]["missed"] > PERSON_MAX_MISSED:
#                     self.tracks.pop(tid, None)
#             return

#         used_det = set()
#         track_ids = list(self.tracks.keys())

#         for tid in track_ids:
#             best_j = -1
#             best_iou = 0.0
#             tbox = self.tracks[tid]["xyxy"]
#             for j in range(len(person_xyxy)):
#                 if j in used_det:
#                     continue
#                 iou = _iou_xyxy(tbox, person_xyxy[j])
#                 if iou > best_iou:
#                     best_iou = iou
#                     best_j = j
#             if best_j >= 0 and best_iou >= PERSON_MATCH_IOU:
#                 self.tracks[tid]["xyxy"] = person_xyxy[best_j].astype(np.float32)
#                 self.tracks[tid]["missed"] = 0
#                 used_det.add(best_j)

#         for j in range(len(person_xyxy)):
#             if j in used_det:
#                 continue
#             tid = self.next_id
#             self.next_id += 1
#             self.tracks[tid] = {"xyxy": person_xyxy[j].astype(np.float32), "missed": 0}

#         for tid in list(self.tracks.keys()):
#             if self.tracks[tid]["missed"] > PERSON_MAX_MISSED:
#                 self.tracks.pop(tid, None)

#     def as_detections(self):
#         if len(self.tracks) == 0:
#             return None
#         tids = sorted(self.tracks.keys())
#         xyxy = np.stack([self.tracks[tid]["xyxy"] for tid in tids], axis=0).astype(np.float32)
#         return sv.Detections(
#             xyxy=xyxy,
#             confidence=np.ones((len(tids),), dtype=np.float32),
#             class_id=np.zeros((len(tids),), dtype=np.int32),
#             tracker_id=np.array(tids, dtype=np.int32),
#         )

# person_annot_tracker = _PersonAnnotTracker()

# # -----------------------
# # Heatvision settings (match reference style)
# # -----------------------
# HEATVISION_COLORMAP = cv2.COLORMAP_JET
# HEAT_BLUR_SIGMA = 2.2

# BG_DARKEN = 0.22
# SUBJECT_GAIN = 1.15
# SUBJECT_GAMMA = 0.75
# GLOW_DILATE = 10
# GLOW_ALPHA = 0.55

# def _soft_silhouette_from_xyxy(xyxy: np.ndarray, H: int, W: int, feather: int = 10):
#     mask = np.zeros((H, W), dtype=np.uint8)
#     if xyxy is None or len(xyxy) == 0:
#         return mask
#     for b in xyxy:
#         x1, y1, x2, y2 = [int(round(v)) for v in b.tolist()]
#         x1 = max(0, min(W - 1, x1))
#         y1 = max(0, min(H - 1, y1))
#         x2 = max(0, min(W - 1, x2))
#         y2 = max(0, min(H - 1, y2))
#         if x2 <= x1 or y2 <= y1:
#             continue
#         cv2.rectangle(mask, (x1, y1), (x2, y2), 255, -1)
#     if feather > 0:
#         k = feather * 2 + 1
#         mask = cv2.GaussianBlur(mask, (k, k), 0)
#     return mask

# # 5. Processing Loop
# frames_generator = sv.get_video_frames_generator(source_path=INPUT_VIDEO_PATH)

# with sv.VideoSink(target_path=OUTPUT_VIDEO_PATH, video_info=video_info) as sink:
#     frame_idx = -1

#     for frame in tqdm(frames_generator, total=video_info.total_frames, desc="Master Logic Render"):
#         frame_idx += 1

#         enhanced_frame = apply_lighting_fix(frame)
#         H, W = enhanced_frame.shape[:2]

#         results = model.track(
#             enhanced_frame,
#             persist=True,
#             verbose=False,
#             conf=0.03,
#             iou=0.4,
#             tracker="botsort.yaml"
#         )[0]

#         detections = sv.Detections.from_ultralytics(results)
#         detections = detections[(detections.class_id == 0) | (detections.class_id == 32)]

#         # HOOP ROI BOOST
#         rx1, ry1, rx2, ry2 = union_gate_roi(HOOP_UPPER_POLYGON, HOOP_LOWER_POLYGON, W, H, pad=30)
#         roi = enhanced_frame[ry1:ry2, rx1:rx2]

#         roi_ball = None
#         if roi.size > 0:
#             roi_res = model.predict(roi, verbose=False, conf=0.01, iou=0.35)[0]
#             roi_det = sv.Detections.from_ultralytics(roi_res)
#             roi_det = roi_det[roi_det.class_id == 32]
#             roi_ball = offset_detections(roi_det, dx=rx1, dy=ry1)

#         full_ball = detections[detections.class_id == 32]
#         merged_ball = concat_dets(full_ball, roi_ball)

#         # Ball: predict -> match -> update
#         pred_xy = ball_track.predict()
#         chosen = pick_best_ball_near_prediction(merged_ball, pred_xy, max_dist=160)

#         if chosen is not None and len(chosen) > 0:
#             cx, cy, bw, bh = det_centroid_wh(chosen)
#             cx = float(cx[0]); cy = float(cy[0])
#             bw = float(bw[0]); bh = float(bh[0])

#             if not ball_track.initialized:
#                 ball_track.init(cx, cy, bw, bh)
#             else:
#                 ball_track.update(cx, cy, bw, bh)
#         else:
#             ball_track.mark_missed()

#         # Virtual ball from Kalman
#         ball_for_zone = None
#         ball_cy = None

#         if ball_track.initialized and ball_track.missed <= MAX_MISSED_FRAMES:
#             bb = ball_track.bbox()
#             if bb is not None:
#                 ball_xyxy = clamp_xyxy(bb, W, H)
#                 ball_cy = float((ball_xyxy[1] + ball_xyxy[3]) / 2.0)
#                 ball_for_zone = sv.Detections(
#                     xyxy=np.array([ball_xyxy], dtype=np.float32),
#                     confidence=np.array([1.0], dtype=np.float32),
#                     class_id=np.array([32], dtype=np.int32),
#                 )
#         else:
#             ball_track.initialized = False

#         ball_seen = ball_for_zone is not None
#         in_upper = upper_zone.trigger(detections=ball_for_zone).any() if ball_seen else False
#         in_lower = lower_zone.trigger(detections=ball_for_zone).any() if ball_seen else False

#         if in_upper:
#             dbg_upper_hits += 1
#         if in_lower:
#             dbg_lower_hits += 1

#         if cooldown > 0:
#             cooldown -= 1

#         # Attempt state machine
#         if attempt["phase"] == "idle":
#             if ball_seen and in_upper and cooldown == 0:
#                 attempt["phase"] = "saw_upper"
#                 attempt["start_frame"] = frame_idx
#                 attempt["start_cy"] = ball_cy

#         elif attempt["phase"] == "saw_upper":
#             if (frame_idx - attempt["start_frame"]) > MAX_TRANSIT_FRAMES:
#                 attempt["phase"] = "idle"
#                 attempt["start_frame"] = None
#                 attempt["start_cy"] = None

#             elif ball_seen and in_lower and cooldown == 0:
#                 down_pixels = ball_cy - float(attempt["start_cy"])
#                 if down_pixels >= MIN_DOWN_PIXELS:
#                     score += 1
#                     cooldown = COOLDOWN_FRAMES
#                     dbg_scores += 1

#                 attempt["phase"] = "idle"
#                 attempt["start_frame"] = None
#                 attempt["start_cy"] = None

#             elif (not ball_seen) and (frame_idx - attempt["start_frame"]) > MISS_RESET_FRAMES:
#                 attempt["phase"] = "idle"
#                 attempt["start_frame"] = None
#                 attempt["start_cy"] = None

#         # ----------------------------
#         # Heatvision base: dark thermal background + heat silhouettes + glow outline
#         # ----------------------------
#         heat_gray = cv2.cvtColor(enhanced_frame, cv2.COLOR_BGR2GRAY)
#         heat_gray = cv2.GaussianBlur(heat_gray, (0, 0), HEAT_BLUR_SIGMA)

#         heat_bg = cv2.applyColorMap(heat_gray, HEATVISION_COLORMAP)
#         annotated_frame = (heat_bg.astype(np.float32) * BG_DARKEN).clip(0, 255).astype(np.uint8)

#         # Persons tracked for continuous trails
#         p_xyxy = detections[detections.class_id == 0].xyxy if len(detections) > 0 else np.zeros((0, 4), dtype=np.float32)
#         if p_xyxy is None:
#             p_xyxy = np.zeros((0, 4), dtype=np.float32)

#         person_annot_tracker.update(p_xyxy)
#         p_tracked = person_annot_tracker.as_detections()

#         # Build silhouette mask for persons + ball
#         person_mask = np.zeros((H, W), dtype=np.uint8)
#         if p_tracked is not None and len(p_tracked) > 0:
#             person_mask = _soft_silhouette_from_xyxy(p_tracked.xyxy, H, W, feather=14)

#         ball_mask = np.zeros((H, W), dtype=np.uint8)
#         if ball_for_zone is not None and len(ball_for_zone) > 0:
#             ball_mask = _soft_silhouette_from_xyxy(ball_for_zone.xyxy, H, W, feather=10)

#         sil = cv2.max(person_mask, ball_mask)

#         hg = heat_gray.astype(np.float32) / 255.0
#         hg = np.power(np.clip(hg, 0.0, 1.0), SUBJECT_GAMMA)
#         hg = (hg * 255.0).clip(0, 255).astype(np.uint8)

#         heat_subject = cv2.applyColorMap(hg, HEATVISION_COLORMAP)
#         heat_subject = (heat_subject.astype(np.float32) * SUBJECT_GAIN).clip(0, 255).astype(np.uint8)

#         sil_f = (sil.astype(np.float32) / 255.0)[..., None]
#         subj_rgb = (heat_subject.astype(np.float32) * sil_f).clip(0, 255).astype(np.uint8)

#         annotated_frame = cv2.add(annotated_frame, subj_rgb)

#         if GLOW_DILATE > 0:
#             k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (GLOW_DILATE * 2 + 1, GLOW_DILATE * 2 + 1))
#             dil = cv2.dilate(sil, k, iterations=1)
#             ring = cv2.subtract(dil, sil)
#             ring = cv2.GaussianBlur(ring, (0, 0), 3.0)

#             ring_f = (ring.astype(np.float32) / 255.0)[..., None]
#             glow_rgb = (heat_subject.astype(np.float32) * ring_f * GLOW_ALPHA).clip(0, 255).astype(np.uint8)
#             annotated_frame = cv2.add(annotated_frame, glow_rgb)

#         # Annotation overlays (kept intact)
#         if p_tracked is not None and len(p_tracked) > 0:
#             annotated_frame = trace_ann.annotate(scene=annotated_frame, detections=p_tracked)
#             annotated_frame = person_ann.annotate(scene=annotated_frame, detections=p_tracked)

#         if ball_for_zone is not None:
#             annotated_frame = ball_ann.annotate(scene=annotated_frame, detections=ball_for_zone)

#         cv2.polylines(annotated_frame, [HOOP_UPPER_POLYGON], True, (0, 255, 255), 2)
#         cv2.polylines(annotated_frame, [HOOP_LOWER_POLYGON], True, (255, 255, 0), 2)

#         # Scoreboard
#         h, w, _ = annotated_frame.shape
#         hud_w, margin = 320, 40
#         overlay = annotated_frame.copy()
#         cv2.rectangle(overlay, (w - hud_w - margin, h - 120), (w - margin, h - 40), (15, 15, 15), -1)
#         cv2.rectangle(overlay, (w - hud_w - margin, h - 120), (w - hud_w - margin + 10, h - 40), (0, 255, 0), -1)
#         annotated_frame = cv2.addWeighted(overlay, 0.7, annotated_frame, 0.3, 0)

#         cv2.putText(annotated_frame, "BASKETBALL PERFORMANCE", (w - hud_w - margin + 25, h - 95),
#                     cv2.FONT_HERSHEY_SIMPLEX, 0.4, (160, 160, 160), 1, cv2.LINE_AA)
#         cv2.putText(annotated_frame, f"PTS: {score}", (w - hud_w - margin + 25, h - 55),
#                     cv2.FONT_HERSHEY_DUPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

#         sink.write_frame(frame=annotated_frame)

# print(f"\n✅ Render Complete! AI Counted: {score}.")
# print("Debug upper hits:", dbg_upper_hits, "lower hits:", dbg_lower_hits, "scores:", dbg_scores)

In [ ]:
# ============================================================
# AUTO HOOP GATES + ROCK-SOLID BALL (KALMAN + REACQUIRE) + ROBUST SCORE
# (NO tracker_id REQUIRED) + NumPy 1.25 FIX + HOOP ROI BOOST
# FULL SELF-CONTAINED BLOCK (includes annotators + lighting fix)
# ============================================================

# 0. Setup annotators (NO LABELS)
person_ann = sv.RoundBoxAnnotator(thickness=2, color=sv.Color.RED)
ball_ann   = sv.RoundBoxAnnotator(thickness=2, color=sv.Color.WHITE)
trace_ann  = sv.TraceAnnotator(thickness=3, trace_length=50)

# 1. Load Model
model = YOLO("yolo11x-seg.pt")

# 1b. Pose model (for dynamic skeleton rendering only)
pose_model = YOLO("yolo11x-pose.pt")

# ----------------------------
# Lighting fix
# ----------------------------
def apply_lighting_fix(frame):
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l2 = clahe.apply(l)
    enhanced = cv2.merge((l2, a, b))
    return cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

# ----------------------------
# Auto hoop gates from rim color
# ----------------------------
def auto_hoop_gates_from_video(video_path: str):
    cap = cv2.VideoCapture(video_path)
    ok, frame = cap.read()
    cap.release()
    if not ok or frame is None:
        raise RuntimeError("Could not read first frame.")

    H, W = frame.shape[:2]
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    lower_orange = np.array([5, 60, 60], dtype=np.uint8)
    upper_orange = np.array([30, 255, 255], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower_orange, upper_orange)

    top_h = int(H * 0.30)
    mask[top_h:, :] = 0
    mask[:top_h, :int(W * 0.35)] = 0

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel, iterations=2)

    ys, xs = np.where(mask > 0)
    if len(xs) < 80:
        raise RuntimeError("Rim not detected. Widen orange HSV thresholds.")

    x1, x2 = int(xs.min()), int(xs.max())
    y1, y2 = int(ys.min()), int(ys.max())

    rim_cx = (x1 + x2) / 2.0
    if abs(rim_cx - (W / 2.0)) > (W * 0.22):
        raise RuntimeError("Detected orange region is not near hoop center. Adjust mask exclusions.")

    rim_w = max(1, x2 - x1)
    pad_x = int(max(10, rim_w * 0.20))

    ux1 = max(0, x1 - pad_x)
    ux2 = min(W - 1, x2 + pad_x)

    uy1 = max(0, y1 - 8)
    uy2 = min(H - 1, y2 + 18)

    ly1 = min(H - 1, y2 + 5)
    ly2 = min(H - 1, y2 + 125)

    upper_poly = np.array([[ux1, uy1], [ux2, uy1], [ux2, uy2], [ux1, uy2]], dtype=int)
    lower_poly = np.array([[ux1, ly1], [ux2, ly1], [ux2, ly2], [ux1, ly2]], dtype=int)
    return upper_poly, lower_poly

video_info = sv.VideoInfo.from_video_path(INPUT_VIDEO_PATH)

try:
    HOOP_UPPER_POLYGON, HOOP_LOWER_POLYGON = auto_hoop_gates_from_video(INPUT_VIDEO_PATH)
    print("✅ Auto hoop gates detected:")
    print("UPPER:", HOOP_UPPER_POLYGON.tolist())
    print("LOWER:", HOOP_LOWER_POLYGON.tolist())
except Exception as e:
    print("⚠️ Auto hoop gates failed, using fallback. Error:", str(e))
    HOOP_UPPER_POLYGON = np.array([[140, 0], [330, 0], [330, 95], [140, 95]], dtype=int)
    HOOP_LOWER_POLYGON = np.array([[150, 85], [320, 85], [320, 215], [150, 215]], dtype=int)

upper_zone = sv.PolygonZone(polygon=HOOP_UPPER_POLYGON)
lower_zone = sv.PolygonZone(polygon=HOOP_LOWER_POLYGON)

# ----------------------------
# Ball stabilizer (Kalman filter)
# ----------------------------
class BallTrack:
    def __init__(self):
        self.kf = cv2.KalmanFilter(4, 2)
        self.kf.transitionMatrix = np.array(
            [[1, 0, 1, 0],
             [0, 1, 0, 1],
             [0, 0, 1, 0],
             [0, 0, 0, 1]], dtype=np.float32
        )
        self.kf.measurementMatrix = np.array(
            [[1, 0, 0, 0],
             [0, 1, 0, 0]], dtype=np.float32
        )
        self.kf.processNoiseCov = np.eye(4, dtype=np.float32) * 1e-2
        self.kf.measurementNoiseCov = np.eye(2, dtype=np.float32) * 2.5e-1
        self.kf.errorCovPost = np.eye(4, dtype=np.float32)

        self.initialized = False
        self.missed = 0
        self.w = 24.0
        self.h = 24.0

    def init(self, cx, cy, w, h):
        self.kf.statePost = np.array([[cx], [cy], [0.0], [0.0]], dtype=np.float32)
        self.kf.statePre = self.kf.statePost.copy()
        self.w = float(w)
        self.h = float(h)
        self.initialized = True
        self.missed = 0

    def predict(self):
        if not self.initialized:
            return None
        pred = self.kf.predict()
        return float(pred[0, 0]), float(pred[1, 0])

    def update(self, cx, cy, w=None, h=None):
        m = np.array([[cx], [cy]], dtype=np.float32)
        self.kf.correct(m)
        if w is not None and h is not None:
            self.w = 0.85 * self.w + 0.15 * float(w)
            self.h = 0.85 * self.h + 0.15 * float(h)
        self.missed = 0

    def mark_missed(self):
        if self.initialized:
            self.missed += 1

    def bbox(self):
        if not self.initialized:
            return None
        cx = float(self.kf.statePost[0, 0])
        cy = float(self.kf.statePost[1, 0])
        x1 = cx - self.w / 2.0
        y1 = cy - self.h / 2.0
        x2 = cx + self.w / 2.0
        y2 = cy + self.h / 2.0
        return np.array([x1, y1, x2, y2], dtype=np.float32)

def det_centroid_wh(dets: sv.Detections):
    xyxy = dets.xyxy
    cx = (xyxy[:, 0] + xyxy[:, 2]) / 2.0
    cy = (xyxy[:, 1] + xyxy[:, 3]) / 2.0
    w = (xyxy[:, 2] - xyxy[:, 0])
    h = (xyxy[:, 3] - xyxy[:, 1])
    return cx, cy, w, h

def pick_best_ball_near_prediction(dets_ball: sv.Detections, pred_xy, max_dist=140):
    if dets_ball is None or len(dets_ball) == 0:
        return None

    if pred_xy is None:
        idx = int(np.argmax(dets_ball.confidence)) if dets_ball.confidence is not None else 0
        return dets_ball[[idx]]

    px, py = pred_xy
    cx, cy, _, _ = det_centroid_wh(dets_ball)
    d = np.sqrt((cx - px) ** 2 + (cy - py) ** 2)
    idx = int(np.argmin(d))

    if float(d[idx]) <= max_dist:
        return dets_ball[[idx]]

    idx2 = int(np.argmax(dets_ball.confidence)) if dets_ball.confidence is not None else 0
    return dets_ball[[idx2]]

def clamp_xyxy(xyxy, W, H):
    x1, y1, x2, y2 = map(float, xyxy.tolist())
    x1 = max(0, min(W - 1, x1))
    y1 = max(0, min(H - 1, y1))
    x2 = max(0, min(W - 1, x2))
    y2 = max(0, min(H - 1, y2))
    return np.array([x1, y1, x2, y2], dtype=np.float32)

def union_gate_roi(upper_poly, lower_poly, W, H, pad=20):
    xs = np.concatenate([upper_poly[:, 0], lower_poly[:, 0]])
    ys = np.concatenate([upper_poly[:, 1], lower_poly[:, 1]])
    x1 = max(0, int(xs.min()) - pad)
    y1 = max(0, int(ys.min()) - pad)
    x2 = min(W - 1, int(xs.max()) + pad)
    y2 = min(H - 1, int(ys.max()) + pad)
    return x1, y1, x2, y2

def offset_detections(dets: sv.Detections, dx: int, dy: int):
    if dets is None or len(dets) == 0:
        return dets
    xyxy = dets.xyxy.copy()
    xyxy[:, [0, 2]] += dx
    xyxy[:, [1, 3]] += dy
    return sv.Detections(
        xyxy=xyxy,
        confidence=dets.confidence.copy() if dets.confidence is not None else None,
        class_id=dets.class_id.copy() if dets.class_id is not None else None,
        tracker_id=dets.tracker_id.copy() if dets.tracker_id is not None else None,
    )

def concat_dets(a: sv.Detections, b: sv.Detections):
    if a is None or len(a) == 0:
        return b
    if b is None or len(b) == 0:
        return a
    return sv.Detections(
        xyxy=np.concatenate([a.xyxy, b.xyxy], axis=0),
        confidence=np.concatenate([a.confidence, b.confidence], axis=0)
        if a.confidence is not None and b.confidence is not None else None,
        class_id=np.concatenate([a.class_id, b.class_id], axis=0)
        if a.class_id is not None and b.class_id is not None else None,
        tracker_id=None,
    )

# -----------------------
# Robust scoring state
# -----------------------
score = 0
COOLDOWN_FRAMES = 45
MAX_TRANSIT_FRAMES = 26
MIN_DOWN_PIXELS = 24
MISS_RESET_FRAMES = 20

cooldown = 0
attempt = {"phase": "idle", "start_frame": None, "start_cy": None}

ball_track = BallTrack()
MAX_MISSED_FRAMES = 30

dbg_upper_hits = 0
dbg_lower_hits = 0
dbg_scores = 0

# -----------------------
# Continuous person boxes + trail even when Ultralytics tracker ids disappear
# (annotation-only lightweight tracker)
# -----------------------
PERSON_MAX_MISSED = 18
PERSON_MATCH_IOU = 0.18

def _iou_xyxy(a, b):
    ax1, ay1, ax2, ay2 = map(float, a)
    bx1, by1, bx2, by2 = map(float, b)
    ix1 = max(ax1, bx1)
    iy1 = max(ay1, by1)
    ix2 = min(ax2, bx2)
    iy2 = min(ay2, by2)
    iw = max(0.0, ix2 - ix1)
    ih = max(0.0, iy2 - iy1)
    inter = iw * ih
    area_a = max(0.0, (ax2 - ax1)) * max(0.0, (ay2 - ay1))
    area_b = max(0.0, (bx2 - bx1)) * max(0.0, (by2 - by1))
    denom = area_a + area_b - inter
    return (inter / denom) if denom > 1e-6 else 0.0

class _PersonAnnotTracker:
    def __init__(self):
        self.next_id = 1
        self.tracks = {}  # id -> {"xyxy": np.array(4), "missed": int}

    def update(self, person_xyxy: np.ndarray):
        for tid in list(self.tracks.keys()):
            self.tracks[tid]["missed"] += 1

        if person_xyxy is None or len(person_xyxy) == 0:
            for tid in list(self.tracks.keys()):
                if self.tracks[tid]["missed"] > PERSON_MAX_MISSED:
                    self.tracks.pop(tid, None)
            return

        used_det = set()
        track_ids = list(self.tracks.keys())

        for tid in track_ids:
            best_j = -1
            best_iou = 0.0
            tbox = self.tracks[tid]["xyxy"]
            for j in range(len(person_xyxy)):
                if j in used_det:
                    continue
                iou = _iou_xyxy(tbox, person_xyxy[j])
                if iou > best_iou:
                    best_iou = iou
                    best_j = j
            if best_j >= 0 and best_iou >= PERSON_MATCH_IOU:
                self.tracks[tid]["xyxy"] = person_xyxy[best_j].astype(np.float32)
                self.tracks[tid]["missed"] = 0
                used_det.add(best_j)

        for j in range(len(person_xyxy)):
            if j in used_det:
                continue
            tid = self.next_id
            self.next_id += 1
            self.tracks[tid] = {"xyxy": person_xyxy[j].astype(np.float32), "missed": 0}

        for tid in list(self.tracks.keys()):
            if self.tracks[tid]["missed"] > PERSON_MAX_MISSED:
                self.tracks.pop(tid, None)

    def as_detections(self):
        if len(self.tracks) == 0:
            return None
        tids = sorted(self.tracks.keys())
        xyxy = np.stack([self.tracks[tid]["xyxy"] for tid in tids], axis=0).astype(np.float32)
        return sv.Detections(
            xyxy=xyxy,
            confidence=np.ones((len(tids),), dtype=np.float32),
            class_id=np.zeros((len(tids),), dtype=np.int32),
            tracker_id=np.array(tids, dtype=np.int32),
        )

person_annot_tracker = _PersonAnnotTracker()

# -----------------------
# Skeleton rendering (pose keypoints)
# -----------------------
POSE_CONF = 0.25
KP_CONF = 0.25
SKELETON_THICKNESS = 3
JOINT_RADIUS = 3

# COCO-17 connections (Ultralytics YOLO pose uses COCO keypoints order)
COCO_EDGES = [
    (5, 7), (7, 9),      # left arm
    (6, 8), (8, 10),     # right arm
    (5, 6),              # shoulders
    (5, 11), (6, 12),    # torso
    (11, 12),            # hips
    (11, 13), (13, 15),  # left leg
    (12, 14), (14, 16),  # right leg
    (0, 1), (0, 2),      # nose to eyes
    (1, 3), (2, 4),      # eyes to ears
    (0, 5), (0, 6),      # nose to shoulders
]

def _draw_pose_skeleton(scene, kpts_xy, kpts_conf, color=(255, 255, 255)):
    # joints
    for i in range(kpts_xy.shape[0]):
        if float(kpts_conf[i]) >= KP_CONF:
            x = int(round(float(kpts_xy[i, 0])))
            y = int(round(float(kpts_xy[i, 1])))
            cv2.circle(scene, (x, y), JOINT_RADIUS, color, -1, cv2.LINE_AA)

    # bones
    for a, b in COCO_EDGES:
        if a >= kpts_xy.shape[0] or b >= kpts_xy.shape[0]:
            continue
        if float(kpts_conf[a]) < KP_CONF or float(kpts_conf[b]) < KP_CONF:
            continue
        x1 = int(round(float(kpts_xy[a, 0])))
        y1 = int(round(float(kpts_xy[a, 1])))
        x2 = int(round(float(kpts_xy[b, 0])))
        y2 = int(round(float(kpts_xy[b, 1])))
        cv2.line(scene, (x1, y1), (x2, y2), color, SKELETON_THICKNESS, cv2.LINE_AA)

def _draw_ball_anchor(scene, xyxy, color=(255, 255, 255), thickness=2):
    x1, y1, x2, y2 = [int(round(v)) for v in xyxy.tolist()]
    if x2 <= x1 or y2 <= y1:
        return
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)
    r = max(3, int(min(x2 - x1, y2 - y1) * 0.35))
    cv2.circle(scene, (cx, cy), r, color, thickness, cv2.LINE_AA)

# 5. Processing Loop
frames_generator = sv.get_video_frames_generator(source_path=INPUT_VIDEO_PATH)

with sv.VideoSink(target_path=OUTPUT_VIDEO_PATH, video_info=video_info) as sink:
    frame_idx = -1

    for frame in tqdm(frames_generator, total=video_info.total_frames, desc="Master Logic Render"):
        frame_idx += 1

        enhanced_frame = apply_lighting_fix(frame)
        H, W = enhanced_frame.shape[:2]

        results = model.track(
            enhanced_frame,
            persist=True,
            verbose=False,
            conf=0.03,
            iou=0.4,
            tracker="botsort.yaml"
        )[0]

        detections = sv.Detections.from_ultralytics(results)
        detections = detections[(detections.class_id == 0) | (detections.class_id == 32)]

        # HOOP ROI BOOST
        rx1, ry1, rx2, ry2 = union_gate_roi(HOOP_UPPER_POLYGON, HOOP_LOWER_POLYGON, W, H, pad=30)
        roi = enhanced_frame[ry1:ry2, rx1:rx2]

        roi_ball = None
        if roi.size > 0:
            roi_res = model.predict(roi, verbose=False, conf=0.01, iou=0.35)[0]
            roi_det = sv.Detections.from_ultralytics(roi_res)
            roi_det = roi_det[roi_det.class_id == 32]
            roi_ball = offset_detections(roi_det, dx=rx1, dy=ry1)

        full_ball = detections[detections.class_id == 32]
        merged_ball = concat_dets(full_ball, roi_ball)

        # Ball: predict -> match -> update
        pred_xy = ball_track.predict()
        chosen = pick_best_ball_near_prediction(merged_ball, pred_xy, max_dist=160)

        if chosen is not None and len(chosen) > 0:
            cx, cy, bw, bh = det_centroid_wh(chosen)
            cx = float(cx[0]); cy = float(cy[0])
            bw = float(bw[0]); bh = float(bh[0])

            if not ball_track.initialized:
                ball_track.init(cx, cy, bw, bh)
            else:
                ball_track.update(cx, cy, bw, bh)
        else:
            ball_track.mark_missed()

        # Virtual ball from Kalman
        ball_for_zone = None
        ball_cy = None

        if ball_track.initialized and ball_track.missed <= MAX_MISSED_FRAMES:
            bb = ball_track.bbox()
            if bb is not None:
                ball_xyxy = clamp_xyxy(bb, W, H)
                ball_cy = float((ball_xyxy[1] + ball_xyxy[3]) / 2.0)
                ball_for_zone = sv.Detections(
                    xyxy=np.array([ball_xyxy], dtype=np.float32),
                    confidence=np.array([1.0], dtype=np.float32),
                    class_id=np.array([32], dtype=np.int32),
                )
        else:
            ball_track.initialized = False

        ball_seen = ball_for_zone is not None
        in_upper = upper_zone.trigger(detections=ball_for_zone).any() if ball_seen else False
        in_lower = lower_zone.trigger(detections=ball_for_zone).any() if ball_seen else False

        if in_upper:
            dbg_upper_hits += 1
        if in_lower:
            dbg_lower_hits += 1

        if cooldown > 0:
            cooldown -= 1

        # Attempt state machine
        if attempt["phase"] == "idle":
            if ball_seen and in_upper and cooldown == 0:
                attempt["phase"] = "saw_upper"
                attempt["start_frame"] = frame_idx
                attempt["start_cy"] = ball_cy

        elif attempt["phase"] == "saw_upper":
            if (frame_idx - attempt["start_frame"]) > MAX_TRANSIT_FRAMES:
                attempt["phase"] = "idle"
                attempt["start_frame"] = None
                attempt["start_cy"] = None

            elif ball_seen and in_lower and cooldown == 0:
                down_pixels = ball_cy - float(attempt["start_cy"])
                if down_pixels >= MIN_DOWN_PIXELS:
                    score += 1
                    cooldown = COOLDOWN_FRAMES
                    dbg_scores += 1

                attempt["phase"] = "idle"
                attempt["start_frame"] = None
                attempt["start_cy"] = None

            elif (not ball_seen) and (frame_idx - attempt["start_frame"]) > MISS_RESET_FRAMES:
                attempt["phase"] = "idle"
                attempt["start_frame"] = None
                attempt["start_cy"] = None

        # ----------------------------
        # Output style: black background + real pose skeletons (dynamic limbs)
        # ----------------------------
        annotated_frame = np.zeros((H, W, 3), dtype=np.uint8)

        # Keep your person continuity tracker (for stable trails)
        p_xyxy = detections[detections.class_id == 0].xyxy if len(detections) > 0 else np.zeros((0, 4), dtype=np.float32)
        if p_xyxy is None:
            p_xyxy = np.zeros((0, 4), dtype=np.float32)
        person_annot_tracker.update(p_xyxy)
        p_tracked = person_annot_tracker.as_detections()

        # Pose inference for dynamic joints
        pose_res = pose_model.predict(enhanced_frame, verbose=False, conf=POSE_CONF, iou=0.45)[0]

        # Draw trail using tracked boxes (kept)
        if p_tracked is not None and len(p_tracked) > 0:
            annotated_frame = trace_ann.annotate(scene=annotated_frame, detections=p_tracked)

        # Draw skeletons from pose keypoints
        # Ultralytics pose: pose_res.keypoints.xy -> (N,17,2), pose_res.keypoints.conf -> (N,17)
        if getattr(pose_res, "keypoints", None) is not None and pose_res.keypoints is not None:
            kxy = pose_res.keypoints.xy
            kcf = pose_res.keypoints.conf

            if kxy is not None and kcf is not None:
                for i in range(len(kxy)):
                    _draw_pose_skeleton(annotated_frame, kxy[i], kcf[i], color=(255, 255, 255))

        # Ball anchor
        if ball_for_zone is not None and len(ball_for_zone) > 0:
            _draw_ball_anchor(annotated_frame, ball_for_zone.xyxy[0], color=(255, 255, 255), thickness=2)

        # Hoop gates still shown
        cv2.polylines(annotated_frame, [HOOP_UPPER_POLYGON], True, (0, 255, 255), 2)
        cv2.polylines(annotated_frame, [HOOP_LOWER_POLYGON], True, (255, 255, 0), 2)

        # Scoreboard
        h, w, _ = annotated_frame.shape
        hud_w, margin = 320, 40
        overlay = annotated_frame.copy()
        cv2.rectangle(overlay, (w - hud_w - margin, h - 120), (w - margin, h - 40), (15, 15, 15), -1)
        cv2.rectangle(overlay, (w - hud_w - margin, h - 120), (w - hud_w - margin + 10, h - 40), (0, 255, 0), -1)
        annotated_frame = cv2.addWeighted(overlay, 0.7, annotated_frame, 0.3, 0)

        cv2.putText(annotated_frame, "BASKETBALL PERFORMANCE", (w - hud_w - margin + 25, h - 95),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (160, 160, 160), 1, cv2.LINE_AA)
        cv2.putText(annotated_frame, f"PTS: {score}", (w - hud_w - margin + 25, h - 55),
                    cv2.FONT_HERSHEY_DUPLEX, 1.2, (255, 255, 255), 2, cv2.LINE_AA)

        sink.write_frame(frame=annotated_frame)

print(f"\n✅ Render Complete! AI Counted: {score}.")
print("Debug upper hits:", dbg_upper_hits, "lower hits:", dbg_lower_hits, "scores:", dbg_scores)

✅ Auto hoop gates detected:
UPPER: [[195, 0], [314, 0], [314, 87], [195, 87]]
LOWER: [[195, 74], [314, 74], [314, 194], [195, 194]]


Master Logic Render:   0%|          | 0/1767 [00:00<?, ?it/s]


✅ Render Complete! AI Counted: 8.
Debug upper hits: 267 lower hits: 187 scores: 8


**Step 5: Download the Result**

Run this final cell to trigger a browser download of your processed video.

In [ ]:
files.download(OUTPUT_VIDEO_PATH)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>